# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re
import nltk
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A55964\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A55964\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\A55964\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Message.db')
df = pd.read_sql_table('InsertTableName',engine)

# X represents the independent variables, or the features, that are used to predict the output variable Y
#X = df[['id','message','original','genre']]
X= df[['message']]
#Y, on the other hand, represents the dependent variable, or the target, that the model is trying to predict.
Y = df.drop(columns=['id','message','original','genre'])
X

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name
3,UN reports Leogane 80-90 destroyed. Only Hospi...
4,"says: west side of Haiti, rest of the country ..."
...,...
26211,The training demonstrated how to enhance micro...
26212,A suitable candidate has been selected and OCH...
26213,"Proshika, operating in Cox's Bazar municipalit..."
26214,"Some 2,000 women protesting against the conduc..."


In [3]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
for col in Y.columns:
     print("Column {} contain values {}".format(Y[col].name,Y[col].unique()))


Column related contain values [1 0 2]
Column request contain values [0 1]
Column offer contain values [0 1]
Column aid_related contain values [0 1]
Column medical_help contain values [0 1]
Column medical_products contain values [0 1]
Column search_and_rescue contain values [0 1]
Column security contain values [0 1]
Column military contain values [0 1]
Column child_alone contain values [0]
Column water contain values [0 1]
Column food contain values [0 1]
Column shelter contain values [0 1]
Column clothing contain values [0 1]
Column money contain values [0 1]
Column missing_people contain values [0 1]
Column refugees contain values [0 1]
Column death contain values [0 1]
Column other_aid contain values [0 1]
Column infrastructure_related contain values [0 1]
Column transport contain values [0 1]
Column buildings contain values [0 1]
Column electricity contain values [0 1]
Column tools contain values [0 1]
Column hospitals contain values [0 1]
Column shops contain values [0 1]
Column ai

In [5]:
Y.dtypes

related                   int64
request                   int64
offer                     int64
aid_related               int64
medical_help              int64
medical_products          int64
search_and_rescue         int64
security                  int64
military                  int64
child_alone               int64
water                     int64
food                      int64
shelter                   int64
clothing                  int64
money                     int64
missing_people            int64
refugees                  int64
death                     int64
other_aid                 int64
infrastructure_related    int64
transport                 int64
buildings                 int64
electricity               int64
tools                     int64
hospitals                 int64
shops                     int64
aid_centers               int64
other_infrastructure      int64
weather_related           int64
floods                    int64
storm                     int64
fire    

In [6]:
#clean related column
# It looks like the "related" category can take 1, 0 or 2 in our dataset.
# we decide to replace it with 1
Y.related.replace(2,1,inplace=True)


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):

    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    words = word_tokenize(text)

    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]

    # Reduce words to their stems
    words = [PorterStemmer().stem(w) for w in words]

    # Reduce words to their root form
    words = [WordNetLemmatizer().lemmatize(w) for w in words]

    return words

In [8]:
#Testing the tokenize function
tokenized = tokenize(X['message'].iloc[3])

print(tokenized)
print(X['message'].iloc[3])

['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
# build pipeline
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier())),
])

In [12]:
pipeline.set_params(clf__estimator__verbose=2)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000223F873D280>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(verbose=2)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model = pipeline.fit(X_train['message'], y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
predicted = model.predict(X_test['message'])
labels = np.unique(predicted)
targets = Y.columns
class_report = classification_report(y_test,predicted,target_names=targets)


# confusion_mat = confusion_matrix(y_test, predicted, labels=labels)
# accuracy = (predicted == y_test).mean()

# print("Labels:", labels)
# print("Confusion Matrix:\n", confusion_mat)
# print("Accuracy:", accuracy)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

In [16]:
print(class_report)

                        precision    recall  f1-score   support

               related       0.84      0.96      0.90      5040
               request       0.84      0.51      0.63      1092
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.70      0.73      2676
          medical_help       0.69      0.07      0.12       515
      medical_products       0.77      0.07      0.13       326
     search_and_rescue       0.71      0.07      0.13       167
              security       0.00      0.00      0.00       110
              military       0.54      0.03      0.07       201
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.39      0.55       404
                  food       0.85      0.62      0.72       718
               shelter       0.84      0.32      0.46       593
              clothing       0.69      0.09      0.16        98
                 money       0.50      

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
#Looking at pipeline parameters to fine tune
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000021537102DC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000021537102DC0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [44]:
parameters = {
    'clf__n_jobs':[2,4,6],
    'clf__estimator__max_depth' : [4,5,6,7,8],
    'clf__estimator__max_features': ['auto','sqrt','log2']
    }
# create grid search object   param_grid=parameters
cv = GridSearchCV(pipeline,param_grid=parameters,verbose=2)

In [45]:
cv.fit(X_train['message'], y_train)

c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\A55

In [46]:
predicted = cv.predict(X_test['message'])
labels = np.unique(predicted)
print(classification_report(y_test,predicted,labels=labels))
print("\nBest parameters: {}".format(cv.best_params_))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      5034
           1       0.85      0.49      0.62      1149

   micro avg       0.84      0.87      0.85      6183
   macro avg       0.84      0.72      0.76      6183
weighted avg       0.84      0.87      0.84      6183
 samples avg       0.73      0.69      0.70      6183


Best parameters: {'clf__n_jobs': 4}


c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


        precision    recall  f1-score   support

           0       0.84      0.95      0.89      5038
           1       0.82      0.46      0.59      1073

   micro avg       0.84      0.86      0.85      6111
   macro avg       0.83      0.71      0.74      6111
weighted avg       0.84      0.86      0.84      6111
 samples avg       0.72      0.69      0.69      6111


Best parameters: {'clf__n_jobs': 6}

   parameters = {
    'clf__n_jobs':[2,4,6],
    'clf__estimator__max_depth' : [4,5,6,7,8],
    'clf__estimator__max_features': ['auto','sqrt','log2']

    }
   
   
   precision    recall  f1-score   support

           0       0.76      1.00      0.86      4976
           1       0.00      0.00      0.00      1164

   micro avg       0.76      0.81      0.78      6140
   macro avg       0.38      0.50      0.43      6140
weighted avg       0.62      0.81      0.70      6140
 samples avg       0.76      0.67      0.70      6140


Best parameters: {'clf__estimator__max_depth': 4, 'clf__estimator__max_features': 'log2', 'clf__n_jobs': 2}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
#After different Grid Search Cross validations we can now set up an improved model

pipeline.set_params(
    clf__estimator__verbose=1,
    clf__estimator__max_features ='log2',
    clf__n_jobs = 6,
    clf__estimator__n_estimators = 500
    )

model_bis = pipeline.fit(X_train['message'], y_train)

c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
predicted = model_bis.predict(X_test['message'])
targets = Y.columns
class_report = classification_report(y_test,predicted,target_names=targets)

c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `z

In [22]:
print(class_report)

                        precision    recall  f1-score   support

               related       0.83      0.97      0.89      5040
               request       0.87      0.45      0.60      1092
                 offer       0.00      0.00      0.00        28
           aid_related       0.82      0.57      0.67      2676
          medical_help       0.33      0.00      0.01       515
      medical_products       0.75      0.02      0.04       326
     search_and_rescue       1.00      0.01      0.02       167
              security       0.00      0.00      0.00       110
              military       0.00      0.00      0.00       201
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.12      0.21       404
                  food       0.92      0.26      0.41       718
               shelter       0.88      0.08      0.14       593
              clothing       0.86      0.06      0.11        98
                 money       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
# Improvement could also mean looking at other types of Multioutput classifiers
# I decided to try the KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier

pipeline_K = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(KNeighborsClassifier())),
])


pipeline_K.set_params(clf__estimator__n_neighbors=3)

model_K = pipeline_K.fit(X_train['message'], y_train)

c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:
predicted_K = model_K.predict(X_test['message'])
class_report = classification_report(y_test,predicted_K,target_names=targets)
print(class_report)

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      5040
               request       0.75      0.15      0.26      1092
                 offer       0.00      0.00      0.00        28
           aid_related       0.72      0.07      0.13      2676
          medical_help       0.30      0.01      0.01       515
      medical_products       0.33      0.01      0.01       326
     search_and_rescue       0.50      0.01      0.01       167
              security       0.00      0.00      0.00       110
              military       0.00      0.00      0.00       201
           child_alone       0.00      0.00      0.00         0
                 water       0.51      0.05      0.09       404
                  food       0.76      0.10      0.18       718
               shelter       0.66      0.07      0.13       593
              clothing       0.75      0.03      0.06        98
                 money       0.00      

c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\A55964\OneDrive - E.ON\Documents\Python Scripts\Udactiy_DataSc_P2\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `z

### 9. Export your model as a pickle file

In [27]:
#saving model
pickle.dump(model_bis, open("model_aleaume.pkl", 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.